In [1]:
from pyspark.sql import SparkSession

import os
from pyspark.sql import SparkSession


# Initialize Spark session with HBase configuration
spark = SparkSession.builder \
    .appName("WriteToHbase11") \
    .config("spark.jars", "/tmp/acryl-spark-lineage-5206f9d-SNAPSHOT.jar,/tmp/hbase-spark-1.1.0-SNAPSHOT.jar") \
    .config("spark.extraListeners", "datahub.spark.DatahubSparkListener")\
    .config("spark.datahub.rest.server", "http://10.208.164.167:8080")\
    .config("spark.datahub.metadata.dataset.materialize", "true")\
    .config("spark.datahub.metadata.dataset.experimental_include_schema_metadata", "true")\
    .getOrCreate()

    #  /tmp/acryl-spark-lineage-5206f9d-SNAPSHOT.jar,/tmp/hbase-spark-1.1.0-SNAPSHOT.jar
    

# .config("spark.hadoop.hbase.zookeeper.quorum", "localhost1") -> oke ăn config này nhé (còn đâu khả năng nó ăn mặc định, hoặc là file habse.xml tỏng classpath)
# .config("spark.hbase.host", "localhost:2182")  -> khong chay 
# print(os.environ["HBASE_HOME"])
# os.environ["HBASE_HOME"] ="/tmp/a" -> không work

spark.sparkContext.setLogLevel("WARN")

24/12/20 17:28:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
data = [('2','Telangana','Chhetri'),
  ('3','Sikkim','Bhutia'),
  ('4','Hyderabad','Shabbir'),
  ('5','Kerala','Vijayan'),
  ('6','Mizoram','Lalpekhlua')
  ]
columns = ["id","city","name"]
goalsDF = spark.createDataFrame(data=data, schema = columns)
goalsDF.show()

# hdfs 
# goalsDF.write.mode("overwrite").parquet("hdfs://hadoop:9000/tmp/test")
# spark.read.parquet("hdfs://hadoop:9000/tmp/test").show()

#write to hbase

# hbase:001:0> create 'testspark', 'cf'
# Created table testspark
# Took 2.3158 seconds                                                                                                                                                                           
# => Hbase::Table - testspark
# hbase:003:0> scan 'testspark'

# sao k co ip hbase ma no van chay dc k nhi :D
# .option("hbase.zookeeper.quorum", "localhost:2182") \

goalsDF.write\
.format("org.apache.hadoop.hbase.spark")\
.option("hbase.columns.mapping","id STRING :key, name STRING cf:name, city STRING cf:city")\
.option("hbase.namespace", "default")\
.option("hbase.table", "testspark")\
.option("hbase.spark.use.hbasecontext", False)\
.save() 


24/12/20 17:28:47 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
24/12/20 17:28:47 INFO SharedState: Warehouse path is 'file:/spark-warehouse'.
24/12/20 17:28:47 INFO ArgumentParser: Couldn't log config from file, will read it from SparkConf
24/12/20 17:28:47 INFO RedshiftVendor: Checking if Redshift classes are available
24/12/20 17:28:47 INFO RedshiftVendor: Checking if Redshift classes are available
24/12/20 17:28:47 INFO RedshiftVendor: Checking if Redshift classes are available
24/12/20 17:28:47 INFO ConsoleTransport: {"eventTime":"2024-12-20T10:28:46.185Z","producer":"https://github.com/OpenLineage/OpenLineage/tree/1.16.0/integration/spark","schemaURL":"https://openlineage.io/spec/2-0-2/OpenLineage.json#/$defs/RunEvent","eventType":"START","run":{"runId":"0193e39c-d97a-780d-a3d1-cf20766b7d77","facets":{"spark_properties":{"_producer":"https://github.com/OpenLineage/OpenLineage/tree/1.16.0/integration/spark","_schemaURL":"h

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


24/12/20 17:28:49 INFO ArgumentParser: Couldn't log config from file, will read it from SparkConf
24/12/20 17:28:49 WARN CompositeMeterRegistry: A MeterFilter is being configured after a Meter has been registered to this registry. All MeterFilters should be configured before any Meters are registered. If that is not possible or you have a use case where it should be allowed, let the Micrometer maintainers know at https://github.com/micrometer-metrics/micrometer/issues/4920. Enable DEBUG level logging on this logger to see a stack trace of the call configuring this MeterFilter.
24/12/20 17:28:49 INFO RedshiftVendor: Checking if Redshift classes are available
24/12/20 17:28:49 INFO RedshiftVendor: Checking if Redshift classes are available
24/12/20 17:28:49 INFO RedshiftVendor: Checking if Redshift classes are available
24/12/20 17:28:49 INFO OpenLineageToDataHub: Emitting lineage: {"eventTime":"2024-12-20T10:28:49.244Z","producer":"https://github.com/OpenLineage/OpenLineage/tree/1.16.0/

24/12/20 17:28:50 INFO PythonRunner: Times: total = 61, boot = 18, init = 43, finish = 0
24/12/20 17:28:50 INFO PythonRunner: Times: total = 62, boot = 20, init = 42, finish = 0
24/12/20 17:28:50 INFO Executor: Finished task 12.0 in stage 2.0 (TID 17). 1789 bytes result sent to driver
24/12/20 17:28:50 INFO Executor: Finished task 6.0 in stage 2.0 (TID 11). 1850 bytes result sent to driver
24/12/20 17:28:50 INFO TaskSetManager: Finished task 12.0 in stage 2.0 (TID 17) in 78 ms on hadoop113 (executor driver) (9/15)
24/12/20 17:28:50 INFO PythonRunner: Times: total = 64, boot = 22, init = 42, finish = 0
24/12/20 17:28:50 INFO TaskSetManager: Finished task 6.0 in stage 2.0 (TID 11) in 80 ms on hadoop113 (executor driver) (10/15)
24/12/20 17:28:50 INFO Executor: Finished task 4.0 in stage 2.0 (TID 9). 1789 bytes result sent to driver
24/12/20 17:28:50 INFO PythonRunner: Times: total = 60, boot = 18, init = 42, finish = 0
24/12/20 17:28:50 INFO TaskSetManager: Finished task 4.0 in stage 2.0

[Stage 3:==========================================>              (15 + 5) / 20]

24/12/20 17:28:51 INFO ZooKeeper: Session: 0x10000666c26003f closed
24/12/20 17:28:51 INFO ClientCnxn: EventThread shut down for session: 0x10000666c26003f
24/12/20 17:28:51 INFO ZooKeeper: Session: 0x10000666c260034 closed
24/12/20 17:28:51 INFO ClientCnxn: EventThread shut down for session: 0x10000666c260034
24/12/20 17:28:51 INFO ZooKeeper: Session: 0x10000666c260036 closed
24/12/20 17:28:51 INFO ClientCnxn: EventThread shut down for session: 0x10000666c260036
24/12/20 17:28:51 INFO ZooKeeper: Session: 0x10000666c260031 closed
24/12/20 17:28:51 INFO ClientCnxn: EventThread shut down for session: 0x10000666c260031
24/12/20 17:28:51 INFO ZooKeeper: Session: 0x10000666c260040 closed
24/12/20 17:28:51 INFO ClientCnxn: EventThread shut down for session: 0x10000666c260040
24/12/20 17:28:51 INFO SparkHadoopMapRedUtil: No need to commit output of task because needsTaskCommit=false: attempt_202412201728505790436547717127108_0011_m_000003_0
24/12/20 17:28:51 INFO SparkHadoopMapRedUtil: No nee

id STRING :key, name STRING cf:name, city STRING cf:city


In [3]:
spark.stop()

24/12/20 17:28:51 INFO PlanUtils: apply method failed with
java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
py4j.ClientServerConne